In [186]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, Concatenate, Lambda
from tensorflow.keras.models import Model
from spektral.layers import GCNConv
import matplotlib.pyplot as plt
import torch
import torch.nn as nn


In [187]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else 'cpu')
print(f'다음 기기로 학습 : {device}')

다음 기기로 학습 : cuda


In [188]:
# 1. 데이터 로드
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

In [189]:
def process_time_features(df):
    df['event_time'] = pd.to_datetime(df['event_time'], errors='coerce')
    df.dropna(subset=['event_time'], inplace=True)
    df = df.sort_values(['epc_code', 'event_time'])
    
    # 시간 차 계산 (초 단위) 및 로그 변환 (0 포함 고려)
    df['time_diff'] = df.groupby('epc_code')['event_time'].diff().dt.total_seconds().fillna(0)
    df['log_time_diff'] = np.log1p(df['time_diff'])
    
    # 시간 관련 파생 피처
    df['hour'] = df['event_time'].dt.hour
    df['day_of_week'] = df['event_time'].dt.dayofweek  # 0: 월요일 ~ 6: 일요일
    df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
    
    # epc_code 그룹 내에서 이벤트 순서 (1부터 시작)
    df['event_order'] = df.groupby('epc_code').cumcount() + 1
    
    return df

In [190]:
def process_time_features(df):
    df['event_time'] = pd.to_datetime(df['event_time'], errors='coerce')
    df.dropna(subset=['event_time'], inplace=True)
    df = df.sort_values(['epc_code', 'event_time'])
    
    # 시간 차 계산 (초 단위) 및 로그 변환 (0 포함 고려)
    df['time_diff'] = df.groupby('epc_code')['event_time'].diff().dt.total_seconds().fillna(0)
    df['log_time_diff'] = np.log1p(df['time_diff'])
    
    # 시간 관련 파생 피처
    df['hour'] = df['event_time'].dt.hour
    df['day_of_week'] = df['event_time'].dt.dayofweek  # 0: 월요일 ~ 6: 일요일
    df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
    
    # epc_code 그룹 내에서 이벤트 순서 (1부터 시작)
    df['event_order'] = df.groupby('epc_code').cumcount() + 1
    
    return df

In [191]:
def encode_categories(df):
    le_hub = LabelEncoder()
    le_event = LabelEncoder()
    df['hub_type_enc'] = le_hub.fit_transform(df['hub_type'])
    df['event_type_enc'] = le_event.fit_transform(df['event_type'])
    
    # 결합 피처 (필요 시 사용)
    df['hub_event_combined'] = df['hub_type'] + '_' + df['event_type']
    le_combined = LabelEncoder()
    df['hub_event_enc'] = le_combined.fit_transform(df['hub_event_combined'])
    
    return df, le_hub, le_event

In [192]:
def build_sequences(df, seq_length=10):
    feature_cols = [
        'log_time_diff', 'hour', 'day_of_week', 'is_weekend',
        'event_type_enc', 'hub_event_enc', 'event_order'
    ]
    hub_index_col = 'hub_type_enc'
    
    sequence_features = []
    hub_indices = []
    
    for epc, group in df.groupby('epc_code'):
        group = group.sort_values('event_time')
        data_features = group[feature_cols].values
        data_hub_indices = group[hub_index_col].values
        if len(data_features) < seq_length:
            continue
        # 슬라이딩 윈도우 방식으로 시퀀스 생성
        for i in range(len(data_features) - seq_length + 1):
            sequence_features.append(data_features[i:i+seq_length])
            hub_indices.append(data_hub_indices[i:i+seq_length])
            
    sequence_features = np.array(sequence_features)
    hub_indices = np.array(hub_indices)
    return sequence_features, hub_indices

In [193]:
def normalize_sequences(sequences):
    num_samples, seq_length, num_features = sequences.shape
    scaler = StandardScaler()
    sequences_flat = sequences.reshape(-1, num_features)
    sequences_scaled_flat = scaler.fit_transform(sequences_flat)
    sequences_scaled = sequences_scaled_flat.reshape(num_samples, seq_length, num_features)
    return sequences_scaled, scaler

In [194]:
def build_hub_graph(df):
    edges = []
    for epc, group in df.groupby('epc_code'):
        group = group.sort_values('event_time')
        hubs = group['hub_type'].values
        for i in range(len(hubs) - 1):
            edges.append((hubs[i], hubs[i+1]))
            
    G = nx.DiGraph()
    for u, v in edges:
        if G.has_edge(u, v):
            G[u][v]['weight'] += 1
        else:
            G.add_edge(u, v, weight=1)
    return G

In [ ]:
def preprocess_pipeline(file_path, seq_length=10):
    df = load_data(file_path)
    df = process_time_features(df)
    df, le_hub, le_event = encode_categories(df)
    df = df.drop(columns=['product_serial', 'product_name'], errors='ignore')
    
    sequence_features, hub_indices = build_sequences(df, seq_length)
    sequence_features_scaled, scaler = normalize_sequences(sequence_features)
    hub_graph = build_hub_graph(df)
    
    return sequence_features_scaled, hub_indices, hub_graph, scaler, le_hub, le_event

# 파일 경로와 시퀀스 길이 설정
file_path = "Dummy_02.csv" 
seq_length = 10            # 한 시퀀스에 포함할 이벤트 개수

# 전처리 파이프라인 실행
sequence_features_scaled, hub_indices, hub_graph, scaler, le_hub, le_event = preprocess_pipeline(file_path, seq_length)

print("전처리된 시퀀스 데이터 shape:", sequence_features_scaled.shape)  
print("전처리된 hub_indices shape:", hub_indices.shape)

# GNN 입력용 그래프 데이터 준비
num_nodes = 6
print("고유 hub_type 수 (num_nodes):", num_nodes)

# 그래프의 노드 피처: 여기서는 단위행렬(One-hot) 사용 (shape: (num_nodes, num_nodes))
graph_nodes_features = np.eye(num_nodes, dtype=np.float32)
# 인접행렬: hub_graph를 이용하여 (shape: (num_nodes, num_nodes))
# 노드 순서를 정렬하여 일관되게 처리합니다.
A = np.eye(num_nodes, dtype=np.float32)

graph_nodes_features = np.expand_dims(graph_nodes_features, axis=0)
A = np.expand_dims(A, axis=0)

# # 3. GNN + LSTM 모델 구성 (Spektral 활용)

# 하이퍼파라미터 설정
sequence_feature_dim = sequence_features_scaled.shape[-1]  # 여기서는 7 (log_time_diff, hour, day_of_week, is_weekend, event_type_enc, hub_event_enc, event_order)
   # GNN으로 얻을 hub 임베딩 차원

# 1. **그래프(GNN) 분기**  
# 그래프 입력: 
# - graph_nodes_input: 각 노드의 피처 (여기서는 one-hot 벡터, shape: (num_nodes,))
# - A_input: 인접행렬 (shape: (num_nodes,))
graph_nodes_input = Input(shape=(num_nodes, num_nodes), batch_size=1, name='graph_nodes_input')
A_input = Input(shape=(num_nodes, num_nodes), batch_size=1, name='A_input')


# GCNConv 레이어를 통해 각 노드의 임베딩 생성  
embedding_dim = 16
gnn_out = GCNConv(embedding_dim, activation='relu', name='gcn_conv')([graph_nodes_input, A_input])
print(gnn_out)
# gnn_out의 shape: (num_nodes, embedding_dim)

# 2. **시퀀스(LSTM) 분기**  
# 시퀀스 데이터 입력 (정규화된 나머지 피처들)
sequence_input = Input(shape=(seq_length, sequence_feature_dim), name='sequence_input')
# 시퀀스 내 각 이벤트에 해당하는 hub_type 인덱스 (정수, shape: (seq_length,))
hub_index_input = Input(shape=(seq_length,), dtype='int32', name='hub_index_input')

In [160]:
# Lambda 레이어를 사용하여, gnn_out(노드 임베딩 행렬)에서 hub_index_input에 해당하는 임베딩을 lookup
def lookup_embeddings(args):
    hub_embeddings, hub_indices = args
    return tf.gather(hub_embeddings, hub_indices)

hub_embeddings_seq = Lambda(lookup_embeddings, name='hub_lookup')([gnn_out, hub_index_input])
# hub_embeddings_seq의 shape: (batch_size, seq_length, embedding_dim)

# 시퀀스의 다른 피처와 hub 임베딩을 concatenate하여 최종 입력 구성
augmented_sequence = Concatenate(axis=-1, name='concat_features')([sequence_input, hub_embeddings_seq])
# 최종 입력 shape: (batch_size, seq_length, sequence_feature_dim + embedding_dim)

# LSTM 기반 오토인코더 구성 (인코더)
encoder = LSTM(128, return_sequences=True, name='encoder_lstm1')(augmented_sequence)
encoder = LSTM(64, return_sequences=False, name='encoder_lstm2')(encoder)
latent = Dense(32, activation='relu', name='latent_dense')(encoder)

# 디코더
decoder_input = RepeatVector(seq_length, name='repeat_vector')(latent)
decoder = LSTM(64, return_sequences=True, name='decoder_lstm1')(decoder_input)
decoder = LSTM(128, return_sequences=True, name='decoder_lstm2')(decoder)

# 재구성 출력 차원은 augmented_sequence와 동일
decoder_output = TimeDistributed(Dense(sequence_feature_dim + embedding_dim), name='time_distributed')(decoder)

# 모델 정의: 총 4개의 입력을 받음
model = Model(
    inputs=[sequence_input, hub_index_input, graph_nodes_input, A_input],
    outputs=decoder_output
)
model.compile(optimizer='adam', loss='mse')
model.summary()

y_train = sequence_features_scaled  # 정상 데이터라 가정

# 모델 학습 (배치 사이즈와 epochs는 필요에 맞게 조정)
history = model.fit(
    x=[sequence_features_scaled, hub_indices, graph_nodes_features, A],
    y=y_train,
    epochs=20,
    batch_size=32
)

# 학습 결과 시각화 (Loss)
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Loss')
plt.show()

loss는 계속 감소하지만 val_loss가 증가하기 시작하면 과적합 신호입니다.